In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, LeaveOneOut, KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [54]:

data = pd.read_pickle('../create_SL_data/data_18months_ua.pkl')

data[['patient_number', 'MR4']]

,patient_number,MR4
0,01,1
1,02,1
2,03,0
3,04,0
4,05,0
5,06,0
6,07,1
7,08,0
8,09,0
9,11,1


In [4]:
data['patient_number'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '11', '12',
       '13', '14', '15', '16', '17', '18', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '32', '33', '34', '35', '36', '37',
       '38'], dtype=object)

In [5]:
X = data.drop(['MR4', 'patient_number'], axis=1)  
y = data['MR4']                                  

patient_numbers = data['patient_number']


## Leave one out nested cross validation

In [24]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.metrics import f1_score, accuracy_score

models = {
    'RandomForest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {'n_estimators': [50, 100, 200, 300, 500], 'max_depth': [3, 5, 10, 50], 'max_features' : ['sqrt', None]}

    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {'n_neighbors': [3, 5, 7, 10], 'weights': ['uniform', 'distance']}
    },
    'LogisticRegression': {
        'model': LogisticRegression(random_state=42, max_iter=1000),
        'params': {'C': [0.1, 1, 10], 'solver': ['liblinear', 'lbfgs']}
    },
    'LDA': {
        'model': LinearDiscriminantAnalysis(),
        'params': {'solver': ['svd', 'lsqr']}
    }
}

feature_importances = {name: pd.DataFrame(np.zeros((X.shape[0], X.shape[1])), columns=X.columns) for name in models if name == 'RandomForest'}
scores = {name: {'f1_scores': [], 'accuracy_scores': [], 'best_params': []} for name in models}

outer_cv = LeaveOneOut()
inner_cv = LeaveOneOut()

for fold_idx, (train_idx, test_idx) in enumerate(outer_cv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    for name, config in models.items():
        grid_search = GridSearchCV(config['model'], config['params'], cv=inner_cv, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        
        y_pred = best_model.predict(X_test)
        f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
        accuracy = accuracy_score(y_test, y_pred)

        scores[name]['f1_scores'].append(f1)
        scores[name]['accuracy_scores'].append(accuracy)
        scores[name]['best_params'].append(grid_search.best_params_)

        if name == 'RandomForest':
            feature_importances[name].iloc[fold_idx, :] = best_model.feature_importances_
        
    print('fold', fold_idx)

results = {}
for name in models:
    average_f1 = np.mean(scores[name]['f1_scores'])
    average_accuracy = np.mean(scores[name]['accuracy_scores'])
    results[name] = {'Average F1 Score': average_f1, 'Average Accuracy': average_accuracy, 'Best Parameters': scores[name]['best_params'][-1]}

    if name == 'RandomForest':
        mean_fi = feature_importances[name].mean(axis=0).sort_values(ascending=False)
        results[name]['Feature Importances'] = mean_fi

for name, result in results.items():
    print(f"{name} Results:")
    for key, value in result.items():
        print(f"{key}: {value}")


fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9
fold 10
fold 11
fold 12
fold 13
fold 14
fold 15
fold 16
fold 17
fold 18
fold 19
fold 20
fold 21
fold 22
fold 23
fold 24
fold 25
fold 26
fold 27
fold 28
fold 29
fold 30
fold 31
fold 32
fold 33
RandomForest Results:
Average F1 Score: 0.4117647058823529
Average Accuracy: 0.4117647058823529
Best Parameters: {'max_depth': 3, 'max_features': 'sqrt', 'n_estimators': 100}
Feature Importances: 90th_158Gd_pSTAT3 Y705_3h_cluster_20                   0.010360
90th_144Nd_pTyr_3h_cluster_25                          0.010016
q90_difference_pre_3h_cluster_27_144Nd_pTyr            0.009612
90th_151Eu_pSTAT3 S727_3h_cluster_26                   0.008578
q90_difference_pre_3h_cluster_6_171Yb_pERK_T202Y204    0.008387
                                                         ...   
q90_difference_pre_3h_cluster_19_173Yb_STAT3tot        0.000000
90th_170Er_pSRC Y418_3h_cluster_21                     0.000000
90th_153Eu_pSTAT1 Y701_pre_clu

In [44]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model1 = LinearDiscriminantAnalysis(solver='svd')
model1.fit(X, y)

coefficients = model1.coef_[0]  

feature_importance = pd.DataFrame(data={'Feature': X.columns, 'Importance': coefficients})

feature_importance['Absolute Importance'] = feature_importance['Importance'].abs()
feature_importance = feature_importance.sort_values(by='Absolute Importance', ascending=False)

feature_importance.head(30)

,Feature,Importance,Absolute Importance
494,90th_142Nd_cCaspase3_3h_cluster_18,0.265054,0.265054
824,90th_172Yb_pS6 S235S236_3h_cluster_12,0.252507,0.252507
826,90th_172Yb_pS6 S235S236_3h_cluster_14,0.235535,0.235535
1596,q90_difference_pre_3h_cluster_0_172Yb_pS6_S235...,0.226756,0.226756
1606,q90_difference_pre_3h_cluster_10_172Yb_pS6_S23...,0.210826,0.210826
1608,q90_difference_pre_3h_cluster_12_172Yb_pS6_S23...,0.204043,0.204043
413,90th_172Yb_pS6 S235S236_pre_cluster_21,-0.203092,0.203092
830,90th_172Yb_pS6 S235S236_3h_cluster_18,0.191092,0.191092
2032,q90_difference_pre_7d_cluster_16_172Yb_pS6_S23...,0.186890,0.186890
1604,q90_difference_pre_3h_cluster_8_172Yb_pS6_S235...,0.183559,0.183559


In [56]:
model1.predict(X)

model1.score(X, y)


0.8235294117647058

In [55]:
data['pred'] = model1.predict(X)

data[['patient_number', 'MR4', 'pred']]

,patient_number,MR4,pred
0,01,1,1
1,02,1,1
2,03,0,0
3,04,0,0
4,05,0,0
5,06,0,0
6,07,1,1
7,08,0,0
8,09,0,1
9,11,1,0


In [8]:
data_pool2 = pd.read_pickle('../create_SL_data_pool2/data_18months_ua_pool2.pkl')
data_pool2

,patient_number,cluster_0_size-diff%_pre_to_3h,cluster_1_size-diff%_pre_to_3h,cluster_2_size-diff%_pre_to_3h,cluster_3_size-diff%_pre_to_3h,cluster_4_size-diff%_pre_to_3h,cluster_5_size-diff%_pre_to_3h,cluster_6_size-diff%_pre_to_3h,cluster_7_size-diff%_pre_to_3h,cluster_8_size-diff%_pre_to_3h,...,q90_difference_pre_7d_cluster_19_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_20_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_21_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_22_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_23_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_24_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_25_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_26_158Gd_pSTAT3_Y705,q90_difference_pre_7d_cluster_27_158Gd_pSTAT3_Y705,MR4
0,39,16.416486,-15.258162,-12.704969,39.847785,-47.416215,29.832593,-40.430970,-6.462319,-20.090784,...,-0.034750,0.229687,0.626132,-0.143142,0.269882,0.127092,-0.414868,-0.162613,0.344658,1
1,40,33.697411,-38.098859,-33.756878,-16.572408,151.489844,5.337323,21.767160,37.769784,108.013309,...,0.162496,0.146796,0.336546,0.141476,0.291373,0.019660,0.473581,0.296404,0.200563,0
2,41,75.056549,29.892304,-13.179238,14.746350,-15.490797,116.753585,-32.000788,20.803497,1.773736,...,0.150947,0.164894,0.102656,-0.010701,0.244866,0.093848,0.049281,0.102392,0.073677,1
3,42,-15.042893,0.351732,476.272447,-49.591160,4.334499,-29.583006,111.763230,-37.308348,80.698489,...,0.217287,0.145196,0.203353,0.312019,0.170165,0.373296,0.028158,0.007622,0.169976,0
4,44,-14.167202,-5.072256,-8.444902,-35.541849,-29.277969,-4.662283,-29.259460,-1.496026,-39.068100,...,-0.138324,0.269469,0.262736,-0.093967,0.256280,0.731239,0.057086,-0.675778,0.042646,1
5,45,-78.603544,-74.292453,-67.766057,-90.645443,-80.858884,-77.694407,-89.154766,-91.366303,-66.321848,...,0.135182,0.199715,-0.054174,-0.007470,0.303452,0.045522,-0.074229,0.019787,0.214555,1
6,54,136.809511,-43.131385,393.671498,18.703976,474.972437,-28.269485,14.847162,-38.888889,341.708543,...,-0.079955,0.216909,-0.031118,0.229611,0.237155,0.264866,-0.325678,-0.115524,-0.180931,0
7,56,69.042896,41.197290,49.608696,22.457699,-13.900000,46.903421,30.276817,195.238095,37.743191,...,0.000000,-0.165315,-0.136707,0.520155,0.653959,-0.102344,0.000000,0.000000,0.747554,0
8,57,147.835751,169.122898,234.382022,79.677250,86.383929,123.604466,176.710930,219.607843,53.720508,...,0.293582,0.331262,0.329223,-0.431109,0.219175,0.000000,0.000000,-0.637334,-0.049941,1


In [9]:
X_test = data_pool2.drop(['MR4', 'patient_number'], axis=1)  
y_test = data_pool2['MR4']  

y_test                               

0    1
1    0
2    1
3    0
4    1
5    1
6    0
7    0
8    1
Name: MR4, dtype: int64

In [16]:
pred = model1.predict(X_test)

In [19]:

patient_numbers_pool2 = data_pool2['patient_number'].astype(int)
patient_numbers_pool2

pred = pd.DataFrame(pred, columns=['pred'])
pred['patient_numbers'] = patient_numbers_pool2
pred

,pred,patient_numbers
0,1,39
1,1,40
2,1,41
3,0,42
4,1,44
5,1,45
6,1,54
7,1,56
8,0,57


In [20]:
response = pd.read_csv('../response/responses_all.csv')
response = response[response['patient_number'].isin(patient_numbers_pool2)]

response = pd.merge(response, pred, left_on='patient_number', right_on='patient_numbers', how='left')
response

,patient_number,patient_id,batch,month,BCRABL,pred,patient_numbers
0,39,4810_00009,5,3,0.070,1,39
1,39,4810_00009,5,6,0.020,1,39
2,39,4810_00009,5,9,0.020,1,39
3,39,4810_00009,5,12,0.007,1,39
4,39,4810_00009,5,15,0.009,1,39
...,...,...,...,...,...,...,...
58,57,4306_00004,6,9,0.025,0,57
59,57,4306_00004,6,12,NaN,0,57
60,57,4306_00004,6,15,NaN,0,57
61,57,4306_00004,6,17,0.007,0,57


In [42]:
import plotly.express as px

response['BCRABL'] = response['BCRABL'].replace(0, 0.001)

symbols = ['circle', 'square', 'diamond', 'cross', 'hourglass', 'triangle-up', 'hexagon', 'star', 'hexagram', 'bowtie']

response['Symbol'] = response['patient_numbers'].apply(lambda x: symbols[x % len(symbols)])


response['BCRABL'] = response['BCRABL'].replace(0, 0.001)
response['Color'] = response['pred'].map({0: 'red', 1: 'green'})

color_discrete_map = {num: 'green' if response.loc[response['patient_numbers'] == num, 'pred'].iloc[0] == 1 else 'red'
                      for num in response['patient_numbers'].unique()}

fig = px.line(
    response,
    x='month',
    y='BCRABL',
    color='patient_numbers',  
    symbol='Symbol',  
    log_y=True,
    title='BCR-ABL% over Time by Patient',
    markers=True,
    color_discrete_map=color_discrete_map
)

fig.update_layout(
    yaxis_title='BCR-ABL%',
    xaxis_title='Test Time (Months)',
    yaxis_type='log',
    legend_title="Patient Number"
)

fig.update_traces(connectgaps=True, line=dict(dash='solid'))

fig.add_hline(y=0.01)

fig.show()
